In [49]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [50]:
seed = 42
iteration = 3000
threshold = 0.055

In [51]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("submission_original.csv")

In [52]:
def split_customer_country_in_dataframe(df, column_name='customer_country'):
    def split_customer_country(customer_country):
        parts = customer_country.split('/')
        if len(parts) >= 3:
            city = parts[1].strip() if parts[1].strip() else 'NaN'
            country = parts[2].strip()
            return city, country
        return 'NaN', 'NaN'

    df[['city', 'country']] = df[column_name].apply(lambda x: pd.Series(split_customer_country(x) if pd.notnull(x) else (None, None)))
    return df

In [53]:
df_train = split_customer_country_in_dataframe(df_train, 'customer_country')
df_test = split_customer_country_in_dataframe(df_test, 'customer_country')

In [54]:
df_train = df_train.drop(columns=['customer_country', 
                                  'customer_country.1'])

In [55]:
df_test = df_test.drop(columns=['customer_country', 
                                  'customer_country.1'])

In [56]:
def label_encoding(series: pd.Series) -> pd.Series:
    my_dict = {}
    series = series.astype(str)
    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [57]:
label_columns = [
    "city",
    "country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

In [58]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

In [59]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size = 0.2,
    shuffle = True,
    random_state = seed,
)

In [60]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[0, 1])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[0, 1])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[0, 1])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))
    
    return confusion, accuracy, precision, recall, F1

In [61]:
x = df_train.drop('is_converted', axis=1)
y = df_train['is_converted'].astype(int)
x_test = df_test.drop(["is_converted", "id"], axis=1)
kf = StratifiedKFold(shuffle=True, random_state=seed)
mean_c = []
mean_a = []
mean_p = []
mean_r = []
mean_f = []
result = []

for idx, (t_index, v_index) in enumerate(kf.split(x, y)):
    x_train , x_valid = x.iloc[t_index], x.iloc[v_index]
    y_train, y_valid = y.iloc[t_index], y.iloc[v_index]
    model_xgb = XGBRegressor(
        n_estimators = iteration,
        eta = 0.05,
        min_child_weight = 50,
        max_depth = 10,
        colsample_bytree = 0.9,
        subsample = 0.9,
        random_state = seed,
        objective = "binary:logistic",
        eval_metric = 'auc',
    )
    model_xgb.fit(x_train.fillna(-1).values, y_train.astype(int).values.reshape(-1,1))
    pred_xgb = model_xgb.predict(x_valid.fillna(-1).values)
    pred_xgb = (pred_xgb > threshold).astype(int)
    pred_xgb
    
    print(f'\n########### {idx} fold validation result ##############\n')
    c, a, p ,r, f = get_clf_eval(y_valid.astype(int), pred_xgb)
    mean_c.append(c)
    mean_a.append(a)
    mean_p.append(p)
    mean_r.append(r)
    mean_f.append(f)
    
    test_pred_xgb = model_xgb.predict(x_test.fillna(-1))
    result.append(test_pred_xgb)

print(f'\n########### result ##############\n')
print(f'\n오차행렬:\n{np.mean(mean_c,axis=0).astype(int)}')
print(f'정확도:{np.round(np.mean(mean_a,axis=0),4)}')
print(f'정밀도:{np.round(np.mean(mean_p,axis=0),4)}')
print(f'재현률:{np.round(np.mean(mean_r,axis=0),4)}')
print(f'f1: {np.round(np.mean(mean_f,axis=0), 4)}')

result = np.mean(result, axis=0)
test_pred_xgb = (result > threshold).astype(int)
print(f'True: {sum(test_pred_xgb)}') # True로 예측된 개수


########### 0 fold validation result ##############

오차행렬:
 [[10091   799]
 [   49   921]]

정확도: 0.9285
정밀도: 0.5355
재현율: 0.9495
F1: 0.6848

########### 1 fold validation result ##############

오차행렬:
 [[10068   822]
 [   44   926]]

정확도: 0.9270
정밀도: 0.5297
재현율: 0.9546
F1: 0.6814

########### 2 fold validation result ##############

오차행렬:
 [[10026   864]
 [   43   927]]

정확도: 0.9235
정밀도: 0.5176
재현율: 0.9557
F1: 0.6715

########### 3 fold validation result ##############

오차행렬:
 [[10056   834]
 [   45   925]]

정확도: 0.9259
정밀도: 0.5259
재현율: 0.9536
F1: 0.6779

########### 4 fold validation result ##############

오차행렬:
 [[10044   845]
 [   55   915]]

정확도: 0.9241
정밀도: 0.5199
재현율: 0.9433
F1: 0.6703

########### result ##############


오차행렬:
[[10057   832]
 [   47   922]]
정확도:0.9258
정밀도:0.5257
재현률:0.9513
f1: 0.6772
True: 1939


In [62]:
df_sub = pd.read_csv("submission_original.csv")
df_sub["is_converted"] = test_pred_xgb

df_sub.to_csv("submission.csv", index=False)
df_sub

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,0
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,1
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,0
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,0
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,13855,0.50,/São Paulo/Brazil,AS,NaN,40292,NaN,Enterprise,10.0,NaN,...,LGESP,NaN,0,0,NaN,NaN,NaN,NaN,97,0
5267,7979,0.25,General / / United States,IT,NaN,47466,NaN,Enterprise,0.0,NaN,...,LGEUS,NaN,0,0,NaN,NaN,NaN,NaN,438,1
5268,12887,0.75,/ OURO BRANCO / Brazil,AS,NaN,46227,Specifier/ Influencer,Enterprise,NaN,NaN,...,LGESP,less than 3 months,0,0,NaN,NaN,NaN,NaN,97,0
5269,17530,0.00,/ / Germany,IT,NaN,45667,End Customer,SMB,NaN,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,NaN,429,0
